In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.jit



In [4]:
import torch
import torch.nn as nn
import torch.optim as optim


# Define the ConvNet model
class OneLayer1dConvNet(nn.Module):
    def __init__(self, in_channels, out_channels, bias=True):
        super(OneLayer1dConvNet, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=1, bias = bias)

    def forward(self, x):
        x = self.conv1(x)
        return x


In [9]:
DEVICE = "cpu"#"cuda:0" if torch.cuda.is_available() else "cpu"
input_channels = 2
num_classes = 3
input_size = 1
num_samples = 1
inputs, targets = generate_data(num_samples=num_samples, input_channels=input_channels, input_size=input_size, num_classes=num_classes)

# Function to generate dummy data for 1D ConvNet
def generate_data_1d(num_samples, input_channels, input_size, num_classes):
    # Create a random input tensor
    input_data = torch.randn(num_samples, input_channels, input_size)
    return input_data

inputs = generate_data_1d(num_samples=num_samples, input_channels=input_channels, input_size=input_size, num_classes=num_classes)

# model = TimingModel(num_classes = 1).to(DEVICE)
out_channels = 1
# model = OneLayerConvNet(in_channels=input_channels, out_channels=out_channels, bias=False).to(DEVICE)
model = OneLayer1dConvNet(in_channels=input_channels, out_channels=out_channels, bias=False).to(DEVICE)
print(inputs.shape)

print(model(inputs).shape)

# Access the weights of conv1 layer
weights = model.conv1.weight
print(weights.shape)
print(weights)

# Define loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)


# Train the model
epoch_max = 1000
for epoch in range(epoch_max):
    inputs = generate_data_1d(num_samples=num_samples, input_channels=input_channels, input_size=input_size, num_classes=num_classes)
    X = inputs
    y = inputs[:,0,:]*0.7 + inputs[:,1,:]*0.3
    
    optimizer.zero_grad()
    outputs = model(X.to(DEVICE))
    loss = criterion(outputs, y.float().to(DEVICE))
    loss.backward()
    optimizer.step()
    if epoch % (epoch_max/10) == 0:
        print('Epoch {}: loss {}'.format(epoch, loss.item()))

weights, X, y

torch.Size([1, 2, 1])
torch.Size([1, 1, 1])
torch.Size([1, 2, 1])
Parameter containing:
tensor([[[ 0.3448],
         [-0.2482]]], requires_grad=True)
Epoch 0: loss 0.1982719600200653
Epoch 100: loss 0.02700117789208889
Epoch 200: loss 0.00019820242596324533
Epoch 300: loss 6.321718046820024e-06
Epoch 400: loss 2.67454114410981e-12
Epoch 500: loss 2.220446049250313e-16
Epoch 600: loss 3.552713678800501e-15
Epoch 700: loss 4.996003610813204e-16
Epoch 800: loss 0.0
Epoch 900: loss 3.552713678800501e-15


(Parameter containing:
 tensor([[[0.7000],
          [0.3000]]], requires_grad=True),
 tensor([[[-0.1930],
          [-0.0095]]]),
 tensor([[-0.1379]]))

In [14]:
model_error, select_error = 0, 0
for _ in range(1000):
    inputs = generate_data_1d(num_samples=num_samples, input_channels=input_channels+2, input_size=input_size, num_classes=num_classes)
    X = inputs.clone()[:,:2,:]
    y = inputs[:,0,:]*0.7 + inputs[:,1,:]*0.3
    
    outputs = model(X.to(DEVICE))
    model.conv1.weight, model.conv1.bias, X, y, outputs
    model_error += criterion(outputs, y.float().to(DEVICE))
    select_error += criterion(inputs[:,0,:]*0.7 + inputs[:,1,:]*0.3, y.float().to(DEVICE))

print(f"model_error: {model_error}, select_error: {select_error}")

model_error: 943.0563354492188, select_error: 0.0


In [15]:
# add noise: 2 batch size, has same noise, can ML result be better? , bias=False, more steps to train
DEVICE = "cpu"#"cuda:0" if torch.cuda.is_available() else "cpu"
input_channels = 2
num_classes = 3
input_size = 2
num_samples = 2
inputs, targets = generate_data(num_samples=num_samples, input_channels=input_channels, input_size=input_size, num_classes=num_classes)

# Function to generate dummy data for 1D ConvNet
def generate_data_1d(num_samples, input_channels, input_size, num_classes):
    # Create a random input tensor
    input_data = torch.randn(num_samples, input_channels, input_size)
    return input_data

inputs = generate_data_1d(num_samples=num_samples, input_channels=input_channels, input_size=input_size, num_classes=num_classes)

# model = TimingModel(num_classes = 1).to(DEVICE)
out_channels = 1
# model = OneLayerConvNet(in_channels=input_channels, out_channels=out_channels, bias=False).to(DEVICE)
model = OneLayer1dConvNet(in_channels=input_channels, out_channels=out_channels, bias=False).to(DEVICE)
print(inputs.shape)

print(model(inputs).shape)

# Access the weights of conv1 layer
weights = model.conv1.weight
print(weights.shape)
print(weights)

# Define loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)


# Train the model
epoch_max = 1000
for epoch in range(epoch_max):
    inputs = generate_data_1d(num_samples=num_samples, input_channels=input_channels+2, input_size=input_size, num_classes=num_classes)
    X = inputs.clone()[:,:2,:]
    X[:,0,:] = inputs[:,0,:] +  inputs[:,2,:]*0.5
    X[:,1,:] = inputs[:,0,:] +  inputs[:,3,:]*0.1
    y = inputs[:,0,:]#*0.7 + inputs[:,1,:]*0.3
    
    optimizer.zero_grad()
    outputs = model(X.to(DEVICE))
    loss = criterion(outputs, y.float().to(DEVICE))
    loss.backward()
    optimizer.step()
    if epoch % (epoch_max/10) == 0:
        print('Epoch {}: loss {}'.format(epoch, loss.item()))

model.conv1.weight, model.conv1.bias, X, y, outputs

torch.Size([2, 2, 2])
torch.Size([2, 1, 2])
torch.Size([1, 2, 1])
Parameter containing:
tensor([[[-0.2731],
         [ 0.6738]]], requires_grad=True)
Epoch 0: loss 0.27235954999923706
Epoch 100: loss 0.7929954528808594
Epoch 200: loss 1.8563162088394165
Epoch 300: loss 0.6015964150428772
Epoch 400: loss 0.4140069782733917
Epoch 500: loss 0.3228446841239929
Epoch 600: loss 0.39978161454200745
Epoch 700: loss 0.3098946809768677
Epoch 800: loss 0.24004150927066803
Epoch 900: loss 0.2562407851219177


(Parameter containing:
 tensor([[[0.0192],
          [0.4718]]], requires_grad=True),
 None,
 tensor([[[ 0.2567,  0.0572],
          [-0.1435, -0.5272]],
 
         [[ 2.3229,  0.2323],
          [ 1.9441, -0.0325]]]),
 tensor([[-0.2877, -0.4252],
         [ 1.9893,  0.1690]]),
 tensor([[[-0.0625, -0.2482]],
 
         [[ 0.9682, -0.0105]]], grad_fn=<ConvolutionBackward0>))

In [16]:
model_error, select_error = 0, 0
for _ in range(1000):
    inputs = generate_data_1d(num_samples=num_samples, input_channels=input_channels+2, input_size=input_size, num_classes=num_classes)
    X = inputs.clone()[:,:2,:]
    X[:,0,:] = inputs[:,0,:] +  inputs[:,2,:]*0.5
    X[:,1,:] = inputs[:,0,:] +  inputs[:,3,:]*0.1
    y = inputs[:,0,:]#*0.7 + inputs[:,1,:]*0.3
    outputs = model(X.to(DEVICE))
    model.conv1.weight, model.conv1.bias, X, y, outputs
    model_error += criterion(outputs, y.float().to(DEVICE))
    select_error += criterion(X[:,1,:], y.float().to(DEVICE))

print(f"model_error: {model_error}, select_error: {select_error}")

model_error: 797.2539672851562, select_error: 10.213024139404297
